# 📊 Проект: Рекомендательная система для социальной сети

## 🎯 Предсказание лайков пользователей
 
**Описание:** Обучение CatBoost модели для предсказания лайков постов пользователями на основе их демографических данных и контента постов.

## Импорт библиотек

In [1]:
import os
import time
import warnings
from io import StringIO
from pathlib import Path

import nltk
import numpy as np
import pandas as pd
import psycopg2
from catboost import CatBoostClassifier
from dotenv import load_dotenv
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.metrics import recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
from sqlalchemy import text

warnings.filterwarnings('ignore')

## 📥 2. Загрузка и исследование данных

### Описание датасетов:
- **user_data.csv**: Демографические данные пользователей (163K записей)
- **post_text.csv**: Тексты постов и их темы (7K записей)  
- **feed_data.csv**: Взаимодействия пользователей с постами (10M записей)

In [2]:
%%time

# Определяем пути
PROJECT_ROOT = Path().resolve().parent
RAW_DATA_DIR = PROJECT_ROOT / "part2-ml-model" / "raw_data"
PROCESSED_DATA_DIR = PROJECT_ROOT / "part2-ml-model" / "processed_data"

user_data = pd.read_csv(RAW_DATA_DIR / "user_data.csv")
post_text = pd.read_csv(RAW_DATA_DIR / "post_text.csv")
feed_data = pd.read_csv(RAW_DATA_DIR / "feed_data_10000000.csv")

CPU times: total: 13.5 s
Wall time: 13.7 s


In [3]:
%%time

display(user_data.info(),
        post_text.info(),
        feed_data.info())

display(user_data,
        post_text,
        feed_data)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    163205 non-null  int64
 1   gender     163205 non-null  int64
 2   age        163205 non-null  int64
 3   country    163205 non-null  str  
 4   city       163205 non-null  str  
 5   exp_group  163205 non-null  int64
 6   os         163205 non-null  str  
 7   source     163205 non-null  str  
dtypes: int64(4), str(4)
memory usage: 10.0 MB
<class 'pandas.DataFrame'>
RangeIndex: 7023 entries, 0 to 7022
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   post_id  7023 non-null   int64
 1   text     7023 non-null   str  
 2   topic    7023 non-null   str  
dtypes: int64(1), str(2)
memory usage: 164.7 KB
<class 'pandas.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 5 columns):
 #   Column     Dtype
---  ------     -

None

None

None

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


,timestamp,user_id,post_id,action,target
0,2021-10-31 15:32:01,94388,3770,view,0
1,2021-11-01 10:12:08,94388,732,view,1
2,2021-11-01 10:13:31,94388,732,like,0
3,2021-11-01 10:13:33,94388,915,view,1
4,2021-11-01 10:14:55,94388,915,like,0
...,...,...,...,...,...
9999995,2021-11-12 06:58:12,54841,6482,view,0
9999996,2021-11-12 07:00:00,54841,2371,view,1
9999997,2021-11-12 07:02:54,54841,2371,like,0
9999998,2021-11-12 07:02:56,54841,982,view,0


CPU times: total: 31.2 ms
Wall time: 37.8 ms


## 🛠️ 3. Предобработка текстовых данных

### Шаги:
1. Загрузка NLTK ресурсов
2. Лемматизация и удаление стоп-слов
3. TF-IDF векторизация
4. PCA для снижения размерности (сохранение 80% дисперсии)

In [4]:
%%time 

# Загрузка NLTK ресурсов (если не скачаны)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))  # текст на английском


def preprocess_text(text):
    """Функция предобработки текста"""
    tokens = word_tokenize(text.lower())  # токенизация и приведение к нижнему регистру
    tokens = [lemmatizer.lemmatize(word) for word in tokens if
              word.isalpha() and word not in stop_words]  # лемматизация
    return " ".join(tokens)


# Применяем предобработку
post_text['processed_text'] = post_text['text'].apply(preprocess_text)

# Применяем TF-IDF
tfidf = TfidfVectorizer(min_df=0.15, max_df=0.85)
tfidf_matrix = tfidf.fit_transform(post_text['processed_text'])  # Векторизация всех документов

# Удаляем колонку 'processed_text'
post_text = post_text.drop(columns=['processed_text'])

# Преобразуем результат в DataFrame для удобства
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

# Уменьшаем размерность до стольких компонент, чтобы
pca = PCA(n_components=0.80)
tfidf_pca = pca.fit_transform(tfidf_df)

print(f"Выбрано {tfidf_pca.shape[1]} компонент PCA, объясненная дисперсия: {sum(pca.explained_variance_ratio_) * 100:.2f}%")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\novos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\novos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\novos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Выбрано 20 компонент PCA, объясненная дисперсия: 80.22%
CPU times: total: 7.33 s
Wall time: 7.98 s


## 🔧 4. Создание признаков

### Для пользователей:
- Нормализация количества пользователей по городам/странам
- One-Hot Encoding для возрастных групп и экспериментальных групп
- Кодирование категориальных переменных (OS, source)

### Для временных меток:
- Извлечение месяца и дня
- Нормализация временных признаков

In [5]:
%%time

# список имён для колонок PCA
col_names = [f'tfidf_pc{num + 1}' for num in range(tfidf_pca.shape[1])]

# Преобразуем в DataFrame
tfidf_pca_df = pd.DataFrame(tfidf_pca, columns=col_names)
post_text = pd.concat([post_text, tfidf_pca_df], axis=1)

# Количество пользователей в каждом городе и стране
user_data['city_count'] = user_data.groupby('city')['user_id'].transform('count')
user_data['country_count'] = user_data.groupby('country')['user_id'].transform('count')

# Кодируем 'os' и 'source' как 0 и 1
user_data['os'] = user_data['os'].map({'Android': 0, 'iOS': 1})
user_data['source'] = user_data['source'].map({'ads': 0, 'organic': 1})

# Разбиваем 'age' на категории и применяем OHE для 'age_group'
age_bins = [0, 18, 20, 22, 25, 30, 35, 40, float('inf')]
age_labels = ['<18', '18_20', '20_22', '22_25', '25_30', '30_35', '35_40', 'gt_40']
user_data['age_group'] = pd.cut(user_data['age'], bins=age_bins, labels=age_labels, right=False)
user_data = pd.get_dummies(user_data, columns=['age_group'], drop_first=True)

# Применяем OHE для 'exp_group'
user_data = pd.get_dummies(user_data, columns=['exp_group'], drop_first=True)

user_data.drop(['age', 'city', 'country'], axis=1, inplace=True)

feed_data['timestamp'] = pd.to_datetime(feed_data['timestamp'])
feed_data.sort_values(by='timestamp', ascending=True, inplace=True)
feed_data['month'] = feed_data['timestamp'].dt.month / 12
feed_data['day'] = feed_data['timestamp'].dt.day / 31
feed_data.drop(['timestamp', 'action'], axis=1, inplace=True)

post_text.drop(['text'], axis=1, inplace=True)
post_text = pd.get_dummies(post_text, columns=['topic'], drop_first=True)

user_data[['city_count', 'country_count']] = StandardScaler().fit_transform(user_data[['city_count', 'country_count']])
user_data = user_data.rename(columns={"city_count": "city_countscaled",
                                      "country_count": "country_countscaled"})

# Преобразование bool в int64 для таблицы user_data
bool_columns_user = [
    'age_group_18_20', 'age_group_20_22', 'age_group_22_25',
    'age_group_25_30', 'age_group_30_35', 'age_group_35_40',
    'age_group_gt_40', 'exp_group_1', 'exp_group_2', 'exp_group_3',
    'exp_group_4'
]

user_data[bool_columns_user] = user_data[bool_columns_user].astype('int64')

# Преобразование bool в int64 для таблицы post_text
bool_columns_post = [
    'topic_covid', 'topic_entertainment', 'topic_movie',
    'topic_politics', 'topic_sport', 'topic_tech'
]

post_text[bool_columns_post] = post_text[bool_columns_post].astype('int64')

CPU times: total: 8.89 s
Wall time: 9.87 s


In [6]:
%%time

display(user_data.info(),
        post_text.info(),
        feed_data.info())

display(user_data,
        post_text,
        feed_data)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              163205 non-null  int64  
 1   gender               163205 non-null  int64  
 2   os                   163205 non-null  int64  
 3   source               163205 non-null  int64  
 4   city_countscaled     163205 non-null  float64
 5   country_countscaled  163205 non-null  float64
 6   age_group_18_20      163205 non-null  int64  
 7   age_group_20_22      163205 non-null  int64  
 8   age_group_22_25      163205 non-null  int64  
 9   age_group_25_30      163205 non-null  int64  
 10  age_group_30_35      163205 non-null  int64  
 11  age_group_35_40      163205 non-null  int64  
 12  age_group_gt_40      163205 non-null  int64  
 13  exp_group_1          163205 non-null  int64  
 14  exp_group_2          163205 non-null  int64  
 15  exp_group_3          163205 

None

None

None

,user_id,gender,os,source,city_countscaled,country_countscaled,age_group_18_20,age_group_20_22,age_group_22_25,age_group_25_30,age_group_30_35,age_group_35_40,age_group_gt_40,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,0,0,-0.492196,0.375419,0,0,0,0,1,0,0,0,0,1,0
1,201,0,0,0,-0.461974,0.375419,0,0,0,0,0,1,0,0,0,0,0
2,202,1,0,0,-0.435580,0.375419,0,0,0,0,0,0,0,0,0,0,1
3,203,0,1,0,2.496407,0.375419,1,0,0,0,0,0,0,1,0,0,0
4,204,0,0,0,-0.480572,0.375419,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,0,1,-0.417118,0.375419,0,0,0,0,0,1,0,0,0,0,1
163201,168549,0,0,1,-0.409050,0.375419,1,0,0,0,0,0,0,0,1,0,0
163202,168550,1,0,1,-0.231818,0.375419,0,0,0,0,0,0,1,0,0,0,1
163203,168551,0,1,1,2.496407,0.375419,0,0,0,0,0,1,0,0,0,1,0


,post_id,tfidf_pc1,tfidf_pc2,tfidf_pc3,tfidf_pc4,tfidf_pc5,tfidf_pc6,tfidf_pc7,tfidf_pc8,tfidf_pc9,...,tfidf_pc17,tfidf_pc18,tfidf_pc19,tfidf_pc20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,-0.218558,0.601688,-0.003158,0.056255,0.385736,-0.339179,0.178276,-0.000519,-0.013779,...,0.001073,0.103103,-0.051594,0.045501,0,0,0,0,0,0
1,2,-0.205009,0.397279,-0.018611,-0.105986,-0.012848,0.034922,0.362120,-0.094890,-0.078602,...,-0.011842,-0.027673,0.127649,0.109420,0,0,0,0,0,0
2,3,-0.226079,0.351299,-0.012396,-0.002268,-0.168320,0.031515,0.210111,-0.029437,0.051842,...,-0.144802,0.108878,0.075442,0.155740,0,0,0,0,0,0
3,4,-0.221130,0.495558,-0.057405,0.098004,-0.013409,0.171581,-0.240566,-0.140888,0.025036,...,-0.125282,0.012668,-0.063527,0.055483,0,0,0,0,0,0
4,5,-0.226907,0.551035,-0.022373,0.179819,0.045989,0.142870,-0.191710,-0.069274,-0.299033,...,-0.053152,0.039702,-0.053253,-0.210970,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,-0.292540,-0.217610,-0.397587,0.098741,-0.021369,-0.028251,-0.066127,0.024365,0.031856,...,0.082184,-0.143099,0.087293,0.011876,0,0,1,0,0,0
7019,7316,-0.270304,-0.438322,-0.335058,0.266671,0.026826,0.065955,0.118212,-0.079184,0.038594,...,0.159207,-0.022844,-0.046375,0.246042,0,0,1,0,0,0
7020,7317,-0.257575,-0.270416,0.536110,0.051263,-0.057736,0.034811,0.142500,-0.110504,-0.018741,...,-0.122173,-0.149795,-0.034521,-0.028334,0,0,1,0,0,0
7021,7318,-0.259425,-0.168820,0.419927,-0.054620,-0.124318,0.017562,0.156697,-0.120087,-0.105272,...,-0.148261,0.159995,-0.136097,-0.119240,0,0,1,0,0,0


,user_id,post_id,target,month,day
2304354,66609,3270,1,0.833333,0.032258
1999320,15471,2810,0,0.833333,0.032258
784528,80305,961,0,0.833333,0.032258
2304355,66609,3270,0,0.833333,0.032258
2304356,66609,4320,0,0.833333,0.032258
...,...,...,...,...,...
6426675,45075,3027,0,1.000000,0.935484
3019649,116395,5771,0,1.000000,0.935484
4359850,53559,587,0,1.000000,0.935484
7435524,109595,1702,0,1.000000,0.935484


CPU times: total: 93.8 ms
Wall time: 98.2 ms


In [7]:
%%time

user_data.to_csv(PROCESSED_DATA_DIR / "processed_user_data.csv", index=False)
post_text.to_csv(PROCESSED_DATA_DIR / "processed_post_textML.csv", index=False)

CPU times: total: 1.66 s
Wall time: 2.47 s


## 🤖 5. Обучение CatBoost модели

### Параметры модели:
- Алгоритм: CatBoostClassifier
- Итерации: 500
- Learning rate: 0.07
- Глубина: 6
- Cat features: 20 категориальных признаков
- Метрика: AUC
- Балансировка классов: включена

In [8]:
%%time

user_post_feed = feed_data.merge(user_data, on="user_id", how="left")
user_post_feed = user_post_feed.merge(post_text, on="post_id", how="left")

CPU times: total: 5.7 s
Wall time: 7.53 s


In [9]:
%%time

display(user_post_feed.info())
display(user_post_feed)

<class 'pandas.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 47 columns):
 #   Column               Dtype  
---  ------               -----  
 0   user_id              int64  
 1   post_id              int64  
 2   target               int64  
 3   month                float64
 4   day                  float64
 5   gender               int64  
 6   os                   int64  
 7   source               int64  
 8   city_countscaled     float64
 9   country_countscaled  float64
 10  age_group_18_20      int64  
 11  age_group_20_22      int64  
 12  age_group_22_25      int64  
 13  age_group_25_30      int64  
 14  age_group_30_35      int64  
 15  age_group_35_40      int64  
 16  age_group_gt_40      int64  
 17  exp_group_1          int64  
 18  exp_group_2          int64  
 19  exp_group_3          int64  
 20  exp_group_4          int64  
 21  tfidf_pc1            float64
 22  tfidf_pc2            float64
 23  tfidf_pc3            float64
 24  tfidf_pc

None

,user_id,post_id,target,month,day,gender,os,source,city_countscaled,country_countscaled,...,tfidf_pc17,tfidf_pc18,tfidf_pc19,tfidf_pc20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,66609,3270,1,0.833333,0.032258,1,0,0,-0.485358,0.375419,...,0.092110,-0.017758,-0.056255,-0.094022,1,0,0,0,0,0
1,15471,2810,0,0.833333,0.032258,1,1,0,-0.444743,0.375419,...,-0.134007,-0.115351,-0.195873,-0.294015,1,0,0,0,0,0
2,80305,961,0,0.833333,0.032258,0,1,0,-0.297596,0.375419,...,-0.024728,0.197110,-0.179695,0.083102,0,0,0,1,0,0
3,66609,3270,0,0.833333,0.032258,1,0,0,-0.485358,0.375419,...,0.092110,-0.017758,-0.056255,-0.094022,1,0,0,0,0,0
4,66609,4320,0,0.833333,0.032258,1,0,0,-0.485358,0.375419,...,0.068804,-0.074063,0.026255,0.114552,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,45075,3027,0,1.000000,0.935484,1,1,0,-0.470315,-2.695570,...,-0.200266,0.017232,0.103761,0.088734,1,0,0,0,0,0
9999996,116395,5771,0,1.000000,0.935484,1,0,1,-0.294451,0.375419,...,0.168430,-0.272016,0.306219,-0.064931,0,0,1,0,0,0
9999997,53559,587,0,1.000000,0.935484,0,0,0,-0.468401,-2.732798,...,-0.011890,0.008316,-0.017139,0.058614,0,1,0,0,0,0
9999998,109595,1702,0,1.000000,0.935484,1,0,1,-0.314007,0.375419,...,0.112863,-0.010708,-0.056650,0.300092,0,0,0,0,1,0


CPU times: total: 46.9 ms
Wall time: 80 ms


In [10]:
%%time

X = user_post_feed.drop(['target'], axis=1)
y = user_post_feed['target']

cat_features = ['gender',
                'os',
                'source',
                'age_group_18_20',
                'age_group_20_22',
                'age_group_22_25',
                'age_group_25_30',
                'age_group_30_35',
                'age_group_35_40',
                'age_group_gt_40',
                'exp_group_1',
                'exp_group_2',
                'exp_group_3',
                'exp_group_4',
                'topic_covid',
                'topic_entertainment',
                'topic_movie',
                'topic_politics',
                'topic_sport',
                'topic_tech']

CPU times: total: 0 ns
Wall time: 1.62 ms


In [11]:
%%time

name = "CatBoostClassifier"

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.07,
    depth=6,
    cat_features=cat_features,
    eval_metric='AUC',
    auto_class_weights='Balanced',
    random_seed=42
)

CPU times: total: 0 ns
Wall time: 213 μs


In [12]:
%%time

X_train, y_train = X.iloc[:8500000], y.iloc[:8500000]
X_test, y_test = X.iloc[8500000:], y.iloc[8500000:]

CPU times: total: 0 ns
Wall time: 415 μs


In [13]:
%%time

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()

# Время обучения
print(round(end_time - start_time, 2))

0:	total: 2.48s	remaining: 20m 39s
1:	total: 4.82s	remaining: 19m 59s
2:	total: 7.24s	remaining: 20m
3:	total: 9.43s	remaining: 19m 29s
4:	total: 11.8s	remaining: 19m 24s
5:	total: 14.1s	remaining: 19m 21s
6:	total: 16.3s	remaining: 19m 11s
7:	total: 18.6s	remaining: 19m 5s
8:	total: 20.8s	remaining: 18m 55s
9:	total: 23.1s	remaining: 18m 50s
10:	total: 25.4s	remaining: 18m 49s
11:	total: 27.6s	remaining: 18m 43s
12:	total: 30s	remaining: 18m 42s
13:	total: 32.3s	remaining: 18m 42s
14:	total: 34.7s	remaining: 18m 41s
15:	total: 37.1s	remaining: 18m 42s
16:	total: 39.5s	remaining: 18m 41s
17:	total: 41.9s	remaining: 18m 42s
18:	total: 44.2s	remaining: 18m 38s
19:	total: 46.6s	remaining: 18m 37s
20:	total: 48.7s	remaining: 18m 30s
21:	total: 51.1s	remaining: 18m 30s
22:	total: 53.4s	remaining: 18m 27s
23:	total: 55.6s	remaining: 18m 23s
24:	total: 58s	remaining: 18m 22s
25:	total: 1m	remaining: 18m 19s
26:	total: 1m 2s	remaining: 18m 15s
27:	total: 1m 4s	remaining: 18m 11s
28:	total: 1m 

In [14]:
%%time

# Сохранение модели
model.save_model("catboost_ml_model.cbm")

CPU times: total: 15.6 ms
Wall time: 15.8 ms


## 📊 6. Оценка качества модели

### Метрики:
- AUC-ROC
- Precision-Recall AUC  
- F1-score
- Accuracy, Precision, Recall

### Важность признаков:
- Анализ наиболее влиятельных признаков на предсказание

In [15]:
import os

from catboost import CatBoostClassifier


def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":
        MODEL_PATH = "/workdir/user_input/model"
    else:
        MODEL_PATH = path
    return MODEL_PATH


def load_models():
    model_path = get_model_path("catboost_ml_model.cbm")
    model = CatBoostClassifier()
    model.load_model(model_path)
    return model

In [16]:
%%time

# Загрузка модели
loaded_model = load_models()

CPU times: total: 0 ns
Wall time: 70.4 ms


In [17]:
%%time

# Таблица для хранения коэффициентов
feature_importance_df = pd.DataFrame(index=X.columns)

# Таблица для хранения метрик
metrics_df = pd.DataFrame(columns=["learn_time",
                                   "train_model_score", "test_model_score",
                                   "train_auc_roc", "test_auc_roc", "train_pr_auc", "test_pr_auc",
                                   "train_f1", "test_f1", "train_accuracy", "test_accuracy",
                                   "train_precision", "test_precision", "train_recall", "test_recall"])

CPU times: total: 0 ns
Wall time: 1.92 ms


In [18]:
%%time

print('Оцениваем модель на трейне')

t0 = time.time()
train_model_score = loaded_model.score(X_train, y_train)
t1 = time.time()
print('\t1\t', round(t1 - t0, 2))

t2 = time.time()
y_train_pred = loaded_model.predict(X_train)
t3 = time.time()
print('\t2\t', round(t3 - t2, 2))

t4 = time.time()
y_train_proba = loaded_model.predict_proba(X_train)[:, 1] if hasattr(loaded_model, 'predict_proba') else y_train_pred
t5 = time.time()
print('\t3\t', round(t5 - t4, 2))

t6 = time.time()
train_auc_roc = roc_auc_score(y_train, y_train_proba)
t7 = time.time()
print('\t4\t', round(t7 - t6, 2))

t8 = time.time()
train_pr_auc = average_precision_score(y_train, y_train_proba)
t9 = time.time()
print('\t5\t', round(t9 - t8, 2))

t10 = time.time()
train_f1 = f1_score(y_train, y_train_pred)
t11 = time.time()
print('\t6\t', round(t11 - t10, 2))

t12 = time.time()
train_accuracy = accuracy_score(y_train, y_train_pred)
t13 = time.time()
print('\t7\t', round(t13 - t12, 2))

t14 = time.time()
train_precision = precision_score(y_train, y_train_pred)
t15 = time.time()
print('\t8\t', round(t15 - t14, 2))

t16 = time.time()
train_recall = recall_score(y_train, y_train_pred)
t17 = time.time()
print('\t9\t', round(t17 - t16, 2))

print('Оценили модель на трейне')
print('Оцениваем модель на тесте')

t18 = time.time()
test_model_score = loaded_model.score(X_test, y_test)
t19 = time.time()
print('\t10\t', round(t19 - t18, 2))

t20 = time.time()
y_test_pred = loaded_model.predict(X_test)
t21 = time.time()
print('\t11\t', round(t21 - t20, 2))

t22 = time.time()
y_test_proba = loaded_model.predict_proba(X_test)[:, 1] if hasattr(loaded_model, 'predict_proba') else y_test_pred
t23 = time.time()
print('\t12\t', round(t23 - t22, 2))

t24 = time.time()
test_auc_roc = roc_auc_score(y_test, y_test_proba)
t25 = time.time()
print('\t13\t', round(t25 - t24, 2))

t26 = time.time()
test_pr_auc = average_precision_score(y_test, y_test_proba)
t27 = time.time()
print('\t14\t', round(t27 - t26, 2))

t28 = time.time()
test_f1 = f1_score(y_test, y_test_pred)
t29 = time.time()
print('\t15\t', round(t29 - t28, 2))

t30 = time.time()
test_accuracy = accuracy_score(y_test, y_test_pred)
t31 = time.time()
print('\t16\t', round(t31 - t30, 2))

t32 = time.time()
test_precision = precision_score(y_test, y_test_pred)
t33 = time.time()
print('\t17\t', round(t33 - t32, 2))

t34 = time.time()
test_recall = recall_score(y_test, y_test_pred)
t35 = time.time()
print('\t18\t', round(t35 - t34, 2))

print('Оценили модель на тесте')
print('Запиcываем метрики в таблицу')

# Формируем DataFrame с нужной структурой
metrics = {
    "model_score": [train_model_score, test_model_score],
    "auc_roc": [train_auc_roc, test_auc_roc],
    "pr_auc": [train_pr_auc, test_pr_auc],
    "f1": [train_f1, test_f1],
    "accuracy": [train_accuracy, test_accuracy],
    "precision": [train_precision, test_precision],
    "recall": [train_recall, test_recall]
}

metrics_df = pd.DataFrame(metrics, index=["train", "test"])

print('Запиcали метрики в таблицу')
print('Запиcываем коэффициенты и важности признаков в таблицу')

importances = loaded_model.get_feature_importance()

# Создаем DataFrame с важностью признаков
feature_importance_df = pd.DataFrame({
    'feature': X_train.columns.tolist() if hasattr(X_train, 'columns') else [f'feature_{i}' 
                                                                             for i in range(len(importances))],
    'importance': importances
})

print('Записали коэффициенты и важности признаков в таблицу')

Оцениваем модель на трейне
	1	 282.42
	2	 283.7
	3	 274.99
	4	 12.9
	5	 10.79
	6	 3.57
	7	 0.91
	8	 3.33
	9	 3.42
Оценили модель на трейне
Оцениваем модель на тесте
	10	 47.91
	11	 47.85
	12	 46.74
	13	 2.03
	14	 1.6
	15	 0.77
	16	 0.16
	17	 0.72
	18	 0.67
Оценили модель на тесте
Запиcываем метрики в таблицу
Запиcали метрики в таблицу
Запиcываем коэффициенты и важности признаков в таблицу
Записали коэффициенты и важности признаков в таблицу
CPU times: total: 12min 40s
Wall time: 17min 4s


In [19]:
%%time

# Вывод результатов
print("Feature Importance Table:")
display(feature_importance_df.sort_values(by='importance', ascending=False))

print("\nModel Metrics Table:")
display(metrics_df)

Feature Importance Table:


,feature,importance
7,city_countscaled,20.969818
8,country_countscaled,8.943775
1,post_id,7.641035
15,age_group_gt_40,6.763001
13,age_group_30_35,6.339306
2,month,6.126927
16,exp_group_1,6.014658
17,exp_group_2,5.859274
14,age_group_35_40,4.519769
44,topic_sport,4.085843



Model Metrics Table:


,model_score,auc_roc,pr_auc,f1,accuracy,precision,recall
train,0.585665,0.646158,0.159957,0.239910,0.585665,0.148273,0.628093
test,0.482189,0.622720,0.171750,0.253409,0.482189,0.153531,0.725150


CPU times: total: 31.2 ms
Wall time: 73.1 ms


## 💾 7. Экспорт обработанных данных в PostgreSQL

### Создаваемые таблицы:
1. `processed_user_data` - обработанные признаки пользователей
2. `processed_post_textML` - TF-IDF + темы постов
3. `processed_feed_data_like` - данные о лайках
4. `clear_post_text` - исходные тексты постов

In [3]:
%%time

processed_user_data = pd.read_csv(PROCESSED_DATA_DIR / "processed_user_data.csv")
processed_post_text = pd.read_csv(PROCESSED_DATA_DIR / "processed_post_textML.csv")
feed_data_like = pd.read_csv(RAW_DATA_DIR / "feed_data_like.csv")
clear_post_text = pd.read_csv(RAW_DATA_DIR / "post_text.csv")

CPU times: total: 9.23 s
Wall time: 9.53 s


In [4]:
%%time

display(processed_user_data.info(),
        processed_post_text.info(),
        feed_data_like.info(),
        clear_post_text.info())

display(processed_user_data,
        processed_post_text,
        feed_data_like,
        clear_post_text)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              163205 non-null  int64  
 1   gender               163205 non-null  int64  
 2   os                   163205 non-null  int64  
 3   source               163205 non-null  int64  
 4   city_countscaled     163205 non-null  float64
 5   country_countscaled  163205 non-null  float64
 6   age_group_18_20      163205 non-null  int64  
 7   age_group_20_22      163205 non-null  int64  
 8   age_group_22_25      163205 non-null  int64  
 9   age_group_25_30      163205 non-null  int64  
 10  age_group_30_35      163205 non-null  int64  
 11  age_group_35_40      163205 non-null  int64  
 12  age_group_gt_40      163205 non-null  int64  
 13  exp_group_1          163205 non-null  int64  
 14  exp_group_2          163205 non-null  int64  
 15  exp_group_3          163205 

None

None

None

None

,user_id,gender,os,source,city_countscaled,country_countscaled,age_group_18_20,age_group_20_22,age_group_22_25,age_group_25_30,age_group_30_35,age_group_35_40,age_group_gt_40,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,0,0,-0.492196,0.375419,0,0,0,0,1,0,0,0,0,1,0
1,201,0,0,0,-0.461974,0.375419,0,0,0,0,0,1,0,0,0,0,0
2,202,1,0,0,-0.435580,0.375419,0,0,0,0,0,0,0,0,0,0,1
3,203,0,1,0,2.496407,0.375419,1,0,0,0,0,0,0,1,0,0,0
4,204,0,0,0,-0.480572,0.375419,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,0,1,-0.417118,0.375419,0,0,0,0,0,1,0,0,0,0,1
163201,168549,0,0,1,-0.409050,0.375419,1,0,0,0,0,0,0,0,1,0,0
163202,168550,1,0,1,-0.231818,0.375419,0,0,0,0,0,0,1,0,0,0,1
163203,168551,0,1,1,2.496407,0.375419,0,0,0,0,0,1,0,0,0,1,0


,post_id,tfidf_pc1,tfidf_pc2,tfidf_pc3,tfidf_pc4,tfidf_pc5,tfidf_pc6,tfidf_pc7,tfidf_pc8,tfidf_pc9,...,tfidf_pc17,tfidf_pc18,tfidf_pc19,tfidf_pc20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,-0.218558,0.601688,-0.003158,0.056255,0.385736,-0.339179,0.178276,-0.000519,-0.013779,...,0.001073,0.103103,-0.051594,0.045501,0,0,0,0,0,0
1,2,-0.205009,0.397279,-0.018611,-0.105986,-0.012848,0.034922,0.362120,-0.094890,-0.078602,...,-0.011842,-0.027673,0.127649,0.109420,0,0,0,0,0,0
2,3,-0.226079,0.351299,-0.012396,-0.002268,-0.168320,0.031515,0.210111,-0.029437,0.051842,...,-0.144802,0.108878,0.075442,0.155740,0,0,0,0,0,0
3,4,-0.221130,0.495558,-0.057405,0.098004,-0.013409,0.171581,-0.240566,-0.140888,0.025036,...,-0.125282,0.012668,-0.063527,0.055483,0,0,0,0,0,0
4,5,-0.226907,0.551035,-0.022373,0.179819,0.045989,0.142870,-0.191710,-0.069274,-0.299033,...,-0.053152,0.039702,-0.053253,-0.210970,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,-0.292540,-0.217610,-0.397587,0.098741,-0.021369,-0.028251,-0.066127,0.024365,0.031856,...,0.082184,-0.143099,0.087293,0.011876,0,0,1,0,0,0
7019,7316,-0.270304,-0.438322,-0.335058,0.266671,0.026826,0.065955,0.118212,-0.079184,0.038594,...,0.159207,-0.022844,-0.046375,0.246042,0,0,1,0,0,0
7020,7317,-0.257575,-0.270416,0.536110,0.051263,-0.057736,0.034811,0.142500,-0.110504,-0.018741,...,-0.122173,-0.149795,-0.034521,-0.028334,0,0,1,0,0,0
7021,7318,-0.259425,-0.168820,0.419927,-0.054620,-0.124318,0.017562,0.156697,-0.120087,-0.105272,...,-0.148261,0.159995,-0.136097,-0.119240,0,0,1,0,0,0


,timestamp,user_id,post_id,action,target
0,2021-12-19 21:58:49,72194,2323,like,0
1,2021-11-20 18:36:14,162059,4190,like,0
2,2021-11-20 18:46:11,162059,2033,like,0
3,2021-11-20 18:52:07,162059,1581,like,0
4,2021-11-20 18:59:02,162059,1172,like,0
...,...,...,...,...,...
8206340,2021-12-19 21:44:12,72194,404,like,0
8206341,2021-12-19 21:50:35,72194,5028,like,0
8206342,2021-12-19 21:51:05,72194,3876,like,0
8206343,2021-12-19 21:53:17,72194,7129,like,0


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


CPU times: total: 109 ms
Wall time: 142 ms


In [5]:
%%time

feed_data_like['timestamp'] = pd.to_datetime(feed_data_like['timestamp'])
feed_data_like.sort_values(by='timestamp', ascending=True, inplace=True)
feed_data_like.drop(['timestamp', 'action', 'target'], axis=1, inplace=True)

CPU times: total: 7.77 s
Wall time: 7.95 s


In [6]:
%%time

feed_data_like.to_csv(PROCESSED_DATA_DIR / "processed_feed_data_like.csv", index=False)

CPU times: total: 8.23 s
Wall time: 8.53 s


In [7]:
%%time

processed_feed_data_like = pd.read_csv(PROCESSED_DATA_DIR / "processed_feed_data_like.csv")

CPU times: total: 2.97 s
Wall time: 3.39 s


In [8]:
%%time

display(processed_user_data.info(),
        processed_post_text.info(),
        processed_feed_data_like.info(),
        clear_post_text.info())

display(processed_user_data,
        processed_post_text,
        processed_feed_data_like,
        clear_post_text)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              163205 non-null  int64  
 1   gender               163205 non-null  int64  
 2   os                   163205 non-null  int64  
 3   source               163205 non-null  int64  
 4   city_countscaled     163205 non-null  float64
 5   country_countscaled  163205 non-null  float64
 6   age_group_18_20      163205 non-null  int64  
 7   age_group_20_22      163205 non-null  int64  
 8   age_group_22_25      163205 non-null  int64  
 9   age_group_25_30      163205 non-null  int64  
 10  age_group_30_35      163205 non-null  int64  
 11  age_group_35_40      163205 non-null  int64  
 12  age_group_gt_40      163205 non-null  int64  
 13  exp_group_1          163205 non-null  int64  
 14  exp_group_2          163205 non-null  int64  
 15  exp_group_3          163205 

None

None

None

None

,user_id,gender,os,source,city_countscaled,country_countscaled,age_group_18_20,age_group_20_22,age_group_22_25,age_group_25_30,age_group_30_35,age_group_35_40,age_group_gt_40,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,0,0,-0.492196,0.375419,0,0,0,0,1,0,0,0,0,1,0
1,201,0,0,0,-0.461974,0.375419,0,0,0,0,0,1,0,0,0,0,0
2,202,1,0,0,-0.435580,0.375419,0,0,0,0,0,0,0,0,0,0,1
3,203,0,1,0,2.496407,0.375419,1,0,0,0,0,0,0,1,0,0,0
4,204,0,0,0,-0.480572,0.375419,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,0,1,-0.417118,0.375419,0,0,0,0,0,1,0,0,0,0,1
163201,168549,0,0,1,-0.409050,0.375419,1,0,0,0,0,0,0,0,1,0,0
163202,168550,1,0,1,-0.231818,0.375419,0,0,0,0,0,0,1,0,0,0,1
163203,168551,0,1,1,2.496407,0.375419,0,0,0,0,0,1,0,0,0,1,0


,post_id,tfidf_pc1,tfidf_pc2,tfidf_pc3,tfidf_pc4,tfidf_pc5,tfidf_pc6,tfidf_pc7,tfidf_pc8,tfidf_pc9,...,tfidf_pc17,tfidf_pc18,tfidf_pc19,tfidf_pc20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,-0.218558,0.601688,-0.003158,0.056255,0.385736,-0.339179,0.178276,-0.000519,-0.013779,...,0.001073,0.103103,-0.051594,0.045501,0,0,0,0,0,0
1,2,-0.205009,0.397279,-0.018611,-0.105986,-0.012848,0.034922,0.362120,-0.094890,-0.078602,...,-0.011842,-0.027673,0.127649,0.109420,0,0,0,0,0,0
2,3,-0.226079,0.351299,-0.012396,-0.002268,-0.168320,0.031515,0.210111,-0.029437,0.051842,...,-0.144802,0.108878,0.075442,0.155740,0,0,0,0,0,0
3,4,-0.221130,0.495558,-0.057405,0.098004,-0.013409,0.171581,-0.240566,-0.140888,0.025036,...,-0.125282,0.012668,-0.063527,0.055483,0,0,0,0,0,0
4,5,-0.226907,0.551035,-0.022373,0.179819,0.045989,0.142870,-0.191710,-0.069274,-0.299033,...,-0.053152,0.039702,-0.053253,-0.210970,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,-0.292540,-0.217610,-0.397587,0.098741,-0.021369,-0.028251,-0.066127,0.024365,0.031856,...,0.082184,-0.143099,0.087293,0.011876,0,0,1,0,0,0
7019,7316,-0.270304,-0.438322,-0.335058,0.266671,0.026826,0.065955,0.118212,-0.079184,0.038594,...,0.159207,-0.022844,-0.046375,0.246042,0,0,1,0,0,0
7020,7317,-0.257575,-0.270416,0.536110,0.051263,-0.057736,0.034811,0.142500,-0.110504,-0.018741,...,-0.122173,-0.149795,-0.034521,-0.028334,0,0,1,0,0,0
7021,7318,-0.259425,-0.168820,0.419927,-0.054620,-0.124318,0.017562,0.156697,-0.120087,-0.105272,...,-0.148261,0.159995,-0.136097,-0.119240,0,0,1,0,0,0


,user_id,post_id
0,156604,573
1,142997,6445
2,122594,4554
3,8663,3837
4,94081,5671
...,...,...
8206340,113194,1629
8206341,84572,6836
8206342,140395,4485
8206343,33284,2093


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


CPU times: total: 125 ms
Wall time: 115 ms


In [9]:
%%time

load_dotenv()
SQLALCHEMY_DATABASE_URL = os.getenv("SQLALCHEMY_DATABASE_URL")

engine = create_engine(SQLALCHEMY_DATABASE_URL)

CPU times: total: 62.5 ms
Wall time: 102 ms


In [10]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        DROP TABLE IF EXISTS processed_user_data;
        DROP TABLE IF EXISTS processed_post_text;
        DROP TABLE IF EXISTS processed_feed_data_like;
        DROP TABLE IF EXISTS clear_post_text;
    """))

CPU times: total: 0 ns
Wall time: 2.83 s


In [11]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE processed_user_data (
            user_id BIGINT,
            gender INT,
            os INT,
            source INT,
            city_countscaled FLOAT,
            country_countscaled FLOAT,
            age_group_18_20 INT,
            age_group_20_22 INT,
            age_group_22_25 INT,
            age_group_25_30 INT,
            age_group_30_35 INT,
            age_group_35_40 INT,
            age_group_gt_40 INT,
            exp_group_1 INT,
            exp_group_2 INT,
            exp_group_3 INT,
            exp_group_4 INT
        );
    """))

CPU times: total: 0 ns
Wall time: 314 ms


In [12]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE processed_post_text (
            post_id BIGINT,
            tfidf_pc1 FLOAT,
            tfidf_pc2 FLOAT,
            tfidf_pc3 FLOAT,
            tfidf_pc4 FLOAT,
            tfidf_pc5 FLOAT,
            tfidf_pc6 FLOAT,
            tfidf_pc7 FLOAT,
            tfidf_pc8 FLOAT,
            tfidf_pc9 FLOAT,
            tfidf_pc10 FLOAT,
            tfidf_pc11 FLOAT,
            tfidf_pc12 FLOAT,
            tfidf_pc13 FLOAT,
            tfidf_pc14 FLOAT,
            tfidf_pc15 FLOAT,
            tfidf_pc16 FLOAT,
            tfidf_pc17 FLOAT,
            tfidf_pc18 FLOAT,
            tfidf_pc19 FLOAT,
            tfidf_pc20 FLOAT,
            topic_covid INT,
            topic_entertainment INT,
            topic_movie INT,
            topic_politics INT,
            topic_sport INT,
            topic_tech INT
        );
    """))

CPU times: total: 15.6 ms
Wall time: 309 ms


In [13]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE processed_feed_data_like (
            user_id BIGINT,
            post_id BIGINT
        );
    """))

CPU times: total: 0 ns
Wall time: 303 ms


In [14]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE clear_post_text (
            post_id BIGINT,
            text TEXT,
            topic TEXT
        );
    """))

CPU times: total: 0 ns
Wall time: 334 ms


In [15]:
%%time

def copy_dataframe(df, table_name, conn_str):
    conn = psycopg2.connect(conn_str)
    cur = conn.cursor()

    buffer = StringIO()
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)

    cur.copy_expert(
        f"COPY {table_name} FROM STDIN WITH CSV",
        buffer
    )

    conn.commit()
    cur.close()
    conn.close()

CPU times: total: 0 ns
Wall time: 12.2 μs


In [16]:
t0 = time.time()
copy_dataframe(processed_user_data, "processed_user_data", SQLALCHEMY_DATABASE_URL)
t1 = time.time()
print(round(t1 - t0, 2))

3.75


In [17]:
t2 = time.time()
copy_dataframe(processed_post_text, "processed_post_text", SQLALCHEMY_DATABASE_URL)
t3 = time.time()
print(round(t3 - t2, 2))

1.99


In [18]:
t4 = time.time()
copy_dataframe(processed_feed_data_like, "processed_feed_data_like", SQLALCHEMY_DATABASE_URL)
t5 = time.time()
print(round(t5 - t4, 2))

16.83


In [19]:
t6 = time.time()
copy_dataframe(clear_post_text, "clear_post_text", SQLALCHEMY_DATABASE_URL)
t7 = time.time()
print(round(t7 - t6, 2))

2.57


In [20]:
%%time

def copy_query_to_df(query, conn_str):
    conn = psycopg2.connect(conn_str)
    cur = conn.cursor()

    buffer = StringIO()
    cur.copy_expert(f"COPY ({query}) TO STDOUT WITH CSV HEADER", buffer)
    buffer.seek(0)

    df = pd.read_csv(buffer)

    cur.close()
    conn.close()
    return df

CPU times: total: 0 ns
Wall time: 10.3 μs


In [21]:
t0 = time.time()
processed_user_data_from_db = copy_query_to_df(
    "SELECT * FROM processed_user_data",
    SQLALCHEMY_DATABASE_URL
)
t1 = time.time()
print(round(t1 - t0, 2))

7.19


In [22]:
t2 = time.time()
processed_post_text_from_db = copy_query_to_df(
    "SELECT * FROM processed_post_text",
    SQLALCHEMY_DATABASE_URL
)
t3 = time.time()
print(round(t3 - t2, 2))

2.09


In [23]:
t4 = time.time()
processed_feed_data_like_from_db = copy_query_to_df(
    "SELECT * FROM processed_feed_data_like LIMIT 100000",
    SQLALCHEMY_DATABASE_URL
)
t5 = time.time()
print(round(t5 - t4, 2))

1.64


In [24]:
t6 = time.time()
clear_post_text_from_db = copy_query_to_df(
    "SELECT * FROM clear_post_text",
    SQLALCHEMY_DATABASE_URL
)
t7 = time.time()
print(round(t7 - t6, 2))

5.58


In [25]:
display(processed_user_data_from_db.info(),
        processed_post_text_from_db.info(),
        processed_feed_data_like_from_db.info(),
        clear_post_text_from_db.info())

display(processed_user_data_from_db,
        processed_post_text_from_db,
        processed_feed_data_like_from_db,
        clear_post_text_from_db)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              163205 non-null  int64  
 1   gender               163205 non-null  int64  
 2   os                   163205 non-null  int64  
 3   source               163205 non-null  int64  
 4   city_countscaled     163205 non-null  float64
 5   country_countscaled  163205 non-null  float64
 6   age_group_18_20      163205 non-null  int64  
 7   age_group_20_22      163205 non-null  int64  
 8   age_group_22_25      163205 non-null  int64  
 9   age_group_25_30      163205 non-null  int64  
 10  age_group_30_35      163205 non-null  int64  
 11  age_group_35_40      163205 non-null  int64  
 12  age_group_gt_40      163205 non-null  int64  
 13  exp_group_1          163205 non-null  int64  
 14  exp_group_2          163205 non-null  int64  
 15  exp_group_3          163205 

None

None

None

None

,user_id,gender,os,source,city_countscaled,country_countscaled,age_group_18_20,age_group_20_22,age_group_22_25,age_group_25_30,age_group_30_35,age_group_35_40,age_group_gt_40,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,0,0,-0.492196,0.375419,0,0,0,0,1,0,0,0,0,1,0
1,201,0,0,0,-0.461974,0.375419,0,0,0,0,0,1,0,0,0,0,0
2,202,1,0,0,-0.435580,0.375419,0,0,0,0,0,0,0,0,0,0,1
3,203,0,1,0,2.496407,0.375419,1,0,0,0,0,0,0,1,0,0,0
4,204,0,0,0,-0.480572,0.375419,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,0,1,-0.417118,0.375419,0,0,0,0,0,1,0,0,0,0,1
163201,168549,0,0,1,-0.409050,0.375419,1,0,0,0,0,0,0,0,1,0,0
163202,168550,1,0,1,-0.231818,0.375419,0,0,0,0,0,0,1,0,0,0,1
163203,168551,0,1,1,2.496407,0.375419,0,0,0,0,0,1,0,0,0,1,0


,post_id,tfidf_pc1,tfidf_pc2,tfidf_pc3,tfidf_pc4,tfidf_pc5,tfidf_pc6,tfidf_pc7,tfidf_pc8,tfidf_pc9,...,tfidf_pc17,tfidf_pc18,tfidf_pc19,tfidf_pc20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,-0.218558,0.601688,-0.003158,0.056255,0.385736,-0.339179,0.178276,-0.000519,-0.013779,...,0.001073,0.103103,-0.051594,0.045501,0,0,0,0,0,0
1,2,-0.205009,0.397279,-0.018611,-0.105986,-0.012848,0.034922,0.362120,-0.094890,-0.078602,...,-0.011842,-0.027673,0.127649,0.109420,0,0,0,0,0,0
2,3,-0.226079,0.351299,-0.012396,-0.002268,-0.168320,0.031515,0.210111,-0.029437,0.051842,...,-0.144802,0.108878,0.075442,0.155740,0,0,0,0,0,0
3,4,-0.221130,0.495558,-0.057405,0.098004,-0.013409,0.171581,-0.240566,-0.140888,0.025036,...,-0.125282,0.012668,-0.063527,0.055483,0,0,0,0,0,0
4,5,-0.226907,0.551035,-0.022373,0.179819,0.045989,0.142870,-0.191710,-0.069274,-0.299033,...,-0.053152,0.039702,-0.053253,-0.210970,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,-0.292540,-0.217610,-0.397587,0.098741,-0.021369,-0.028251,-0.066127,0.024365,0.031856,...,0.082184,-0.143099,0.087293,0.011876,0,0,1,0,0,0
7019,7316,-0.270304,-0.438322,-0.335058,0.266671,0.026826,0.065955,0.118212,-0.079184,0.038594,...,0.159207,-0.022844,-0.046375,0.246042,0,0,1,0,0,0
7020,7317,-0.257575,-0.270416,0.536110,0.051263,-0.057736,0.034811,0.142500,-0.110504,-0.018741,...,-0.122173,-0.149795,-0.034521,-0.028334,0,0,1,0,0,0
7021,7318,-0.259425,-0.168820,0.419927,-0.054620,-0.124318,0.017562,0.156697,-0.120087,-0.105272,...,-0.148261,0.159995,-0.136097,-0.119240,0,0,1,0,0,0


,user_id,post_id
0,156604,573
1,142997,6445
2,122594,4554
3,8663,3837
4,94081,5671
...,...,...
99995,137312,1428
99996,130512,1449
99997,95202,5100
99998,2977,6657


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7314,Although the likeliness of someone focusing on...,movie
7019,7315,"OK, I would not normally watch a Farrelly brot...",movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


## 🎯 8. Выводы и результаты

### Ключевые результаты:
1. **Качество модели (ML)**: AUC-ROC = 0.62-0.65
2. **Качество модели (prod)**: HitRate@5 = 0.600
3. **Среднее время работы одного запроса:** 0,5 секунд